In [21]:
!pip install git+https://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-9i1ea382
  Running command git clone -q https://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-9i1ea382
fatal: destination path 'nvcc4jupyter' already exists and is not an empty directory.
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin



Considere que desejamos gerar um array B através de um array A de mesmo tamanho, de modo que, cada elemento `B[i]` é dado pela soma do elemento `A[i]` mais os `r` elementos anteriores à posição `i` em A mais os `r` elementos posteriores, ou seja:
```
for(j=i-r; j<=i+r; j++)
    B[i]+=A[j]
```
O código dado a seguir já faz o cálculo do array B (armazenado em `bHost`) de forma sequencial. Escreva um kernel CUDA, bem como todo o restante do código necessário para executá-lo, de modo a permitir o cálculo de B de forma paralela.

As posições inválidas no início e final do array A devem ser simplesmente descartadas durante a soma, ou seja, para cálculo de `B[10]` com `r=32`, o resultado será a soma dos elementos de `A[0]` até `A[42]`.

Se atente para os seguinte requisitos:
* O cálculo dos elementos de B deve ser realizado de forma paralela;
* A memória compartilhada da GPU deve ser utilizada de forma a deixar a operação mais eficiente;
* O código deve tratar corretamente arrays grandes divididos em mais de um bloco.

Dicas:
* O código abaixo utiliza a constante `RANGE` para definir o valor de `r` e a constante `BLOCKSIZE` para definir o tamanho de cada bloco do grid;
* Por simplicidade, você pode considerar que o tamanho do array A será sempre múltiplo de `BLOCKSIZE`;
* Considere também que o tamanho de `r` será no máximo igual à metade de `BLOCKSIZE`, ou seja, o array alocado na memória compartilhada terá tamanho igual a `2*BLOCKSIZE`, no máximo.
* A estratégia utilizada pela versão sequencial de testar cada posição (para descobrir se é válida) não é ótima. Muitas comparações desnecessárias são realizadas ao calcular as posições intermediárias. Na versão paralela, é melhor preecher as posições inválidas com o valor 0 (zero) ao copiar os dados para a memória compartilhada.

In [23]:
%%gpu
#include<stdio.h>
#include<cuda.h>

#define BLOCKSIZE 1024  //threads por bloco
#define RANGE 256       //deslocamento utilizado para cálculo de B

void initArray(int *a, int n) {
    for (int i =0; i<n; ++i) 
        a[i]=i;
}

void printArray(int *a, int n) {
    int maxPrint=20;
    for (int i =0; i<(n>maxPrint ? maxPrint : n); ++i) 
        printf("%3d ",a[i]);
    if (n>maxPrint) printf("... (array truncado)");
    printf("\n");
}

bool checkArray(int *a, int *b, int n) {
    for (int i =0; i<n; ++i) 
        if (a[i]!=b[i])
            return false;
    return true;
}

void execHost(int *A, int n, int *B) {
    for(int i=0; i<n; i++) {
        int soma=0;
        for(int j=i-RANGE; j<=i+RANGE; j++)
            if(j >= 0 && j<n)
                soma+=A[j];
        B[i]=soma;
    }
}

__global__
void execGPU(int *A, int n, int *B) {
    __shared__ int sA[2 * BLOCKSIZE];

    int inicioBloco = blockIdx.x * blockDim.x;
    int fimBloco = (blockIdx.x + 1) * blockDim.x - 1;
    int idRelativo = threadIdx.x;
    int idGeral = blockIdx.x * blockDim.x + idRelativo;

    sA[idRelativo + RANGE] = A[idGeral];
    
    if(idGeral - RANGE < 0)
        sA[idRelativo] = 0;
    else if(idGeral - RANGE < inicioBloco)
        sA[idRelativo] = A[idGeral - RANGE];
    
    if(idGeral + RANGE >= n)
        sA[idRelativo + 2 * RANGE] = 0;
    else if(idGeral + RANGE > fimBloco)
        sA[idRelativo + 2 * RANGE] = A[idGeral + RANGE];

    __syncthreads();

    int soma = 0;
        
    for(int i = idRelativo; i <= idRelativo + 2 * RANGE; i++)
        soma += sA[i];
        
    B[idGeral] = soma;
}

int main() {
    int size=1<<20;             //tamanho dos arrays A e B
    int dsize=size*sizeof(int); //tamanho dos dados
    int *a;
    int *aGPU;
    int *b;                     //array B a ser calculado na GPU
    int *bGPU;
    int *bHost;                 //array B calculado na CPU
    a=(int*)malloc(dsize);
    b=(int*)malloc(dsize);
    bHost=(int*)malloc(dsize);
    
    initArray(a,size);
    printf("    A: ");
    printArray(a,size);

    cudaMalloc((void **) &aGPU, dsize);
    cudaMalloc((void **) &bGPU, dsize);
    cudaMemcpy(aGPU, a, dsize, cudaMemcpyHostToDevice);
    printf("B-GPU: ");
    execGPU<<<size/BLOCKSIZE,BLOCKSIZE>>>(aGPU, size, bGPU);
    cudaMemcpy(b, bGPU, dsize, cudaMemcpyDeviceToHost);
    printArray(b,size);
    cudaFree(aGPU);
    cudaFree(bGPU); 

    printf("B-CPU: ");
    execHost(a, size, bHost);
    printArray(bHost,size);

    if (checkArray(b,bHost,size))
        printf("Resultados iguais\n");
    else
        printf("Resultados diferentes\n");

    free(bHost);
    free(a);
    free(b);

    return 0;
}

    A:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 ... (array truncado)
B-GPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
B-CPU: 32896 33153 33411 33670 33930 34191 34453 34716 34980 35245 35511 35778 36046 36315 36585 36856 37128 37401 37675 37950 ... (array truncado)
Resultados iguais

